# Time series

* https://docs.scipy.org/doc/scipy/reference/generated/scipy.fft.fft.html
* https://docs.scipy.org/doc/scipy/tutorial/fft.html

* y1 - pure white noise
* y2 - random walk noise
* y3 - a linear drift (with noise)
* y4 - oscaillating function, with noise
* y5 - secret combination of the above

y4 oscillates accoring to

$$
  y_4(t) = A_1 \cos(2\pi f_1 t) + A_2 \sin(2\pi f_2 t)
$$


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftshift, fftfreq

In [ ]:
filename = "timeseries.csv"
data = np.genfromtxt(filename, delimiter=",", skip_header=1)

[t, y1, y2, y3, y4, y5] = [data[:, i] for i in range(6)]

In [ ]:
plt.plot(t, y1, label="y1")
plt.plot(t, y2, label="y2")
plt.plot(t, y3, label="y3")
plt.plot(t, y4, label="y4")
plt.plot(t, y5, label="y5")
plt.legend()
plt.show()

### Histogram

In [ ]:
fig, [ax1, ax2] = plt.subplots(ncols=2, sharex=True, sharey=True)

labels = ["white", "randWalk", "drift", "osc", "mix"]
for i, y in enumerate([y1, y2, y3, y4, y5]):
    name = labels[i]
    tax = ax1 if name == "white" or name == "osc" else ax2
    tax.hist(y, bins=20, density=True, histtype="step", color=f"C{i}", lw=2, label=name)
    tax.hist(y, bins=20, density=True, histtype="stepfilled", color=f"C{i}", alpha=0.25)
    tax.set_yscale("log")
    tax.legend()
plt.show()

### Fourier Transform

In [ ]:
ft = fft(y4)

dt = t[1] - t[0]

f = fftfreq(len(t), d=dt)


plt.plot(f, ft.real, "x-", label="real")
plt.plot(f, ft.imag, ".-", label="imag")
plt.ylabel("FT(y)")
plt.xlabel("f (Hz)")
plt.legend()
plt.show()

### Use 'fftshift' to properly center on 0

In [ ]:
ft = fftshift(fft(y4))
f = fftshift(fftfreq(t.size, d=dt))

plt.plot(f, ft.real, "x-", label="real")
plt.plot(f, ft.imag, ".-", label="imag")
# plt.xlim(0)
plt.ylabel("FT(y)")
plt.xlabel("f (Hz)")
plt.legend()
plt.show()

### Amplitude spectrum from DFT:

$$
  A(f) = \frac{2}{N} |FT(y)|
$$

In [ ]:
ft = fftshift(fft(y4))
freq = fftshift(fftfreq(y4.size, d=dt))


amp = 2.0 * np.abs(ft) / len(t)

plt.plot(freq, amp, "-x", label="amplitude")
plt.xlim(0)
plt.ylabel("Amplitude")
plt.xlabel("f (Hz)")
plt.legend()
plt.show()

### Zoom in, with inset

In [ ]:
from matplotlib import patches

fig, ax1 = plt.subplots()

ft = fftshift(fft(y4))
freq = fftshift(fftfreq(y4.size, d=dt))
amp = 2.0 * np.abs(ft) / len(t)


plt.plot(freq, amp, "-", label="amplitude")
plt.xlim(0)
plt.ylabel("Amplitude")
plt.xlabel("f (Hz)")
plt.legend()

# Define area to zoom in on
x0 = 0.5
dx = 3.0
y0 = 0.0
dy = 1.08

ax1.add_patch(plt.Rectangle((x0, y0), dx, dy, ls="-", fc="None", ec="grey"))


ax2 = fig.add_axes([0.3, 0.4, 0.5, 0.34])
plt.setp(ax2.spines.values(), ls="-", color="grey")
ax2.plot(freq, amp, "-o")
ax2.axhline(y=1.0, color="r", linestyle="dotted", label="$f_1$ actual")
ax2.axvline(x=4.0 / 3, color="r", linestyle="dotted")
ax2.axhline(y=0.5, color="g", linestyle="dashdot", label="$f_2$ actual")
ax2.axvline(x=7.0 / 3, color="g", linestyle="dashdot")
ax2.set_xlim(x0, dx)
ax2.legend()

con1 = patches.ConnectionPatch(
    xyA=(x0, y0),  # actual position in full plot
    coordsA=ax1.transData,
    xyB=(0.0, 0.0),  # relative position on inset plot
    coordsB=ax2.transAxes,
    color="grey",
    linestyle="dotted",
    linewidth=0.5,
)
fig.add_artist(con1)

con3 = patches.ConnectionPatch(
    xyA=(x0 + dx, y0),
    coordsA=ax1.transData,
    xyB=(1.0, 0.0),
    coordsB=ax2.transAxes,
    color="grey",
    ls="dotted",
    lw=0.5,
)
fig.add_artist(con3)

fig.suptitle("Zoom in, with inset", fontsize=15)
plt.show()

In [ ]:
[a1, a2, a3, a4, a5] = [
    2.0 * np.abs(fftshift(fft(y))) / len(t) for y in [y1, y2, y3, y4, y5]
]

f = fftshift(fftfreq(t.size, d=dt))

plt.plot(f, a1, label="White")
plt.plot(f, a2, label="Random Walk")
plt.plot(f, a3, label="Drift")
plt.plot(f, a4, label="Oscillate")
plt.plot(f, a5, label="Combination")
plt.xlim(0, 8)
plt.ylabel("Amplitude")
plt.xlabel("f (Hz)")
plt.legend()
plt.show()